In [ ]:
def ewd():  
    """Apply Equal Width Discretization (EWD) to the training data to determine variances"""
    
    df_train = np.stack((x_train, y_train)).T
    np.matrix(df_train)
    df_train = df_train[np.argsort(df_train[:,0])]
    cuts1, cuts2 = pd.cut(df_train[:,0], nbins, retbins=True)
    
    return df_train, cuts1, cuts2

def binned_statistics():
    """Find the mean, variance and number of counts within the bins described by ewd"""
    
    df_train, cuts1, cuts2 = ewd()
    mean, edges, binnum = scipy.stats.binned_statistic(df_train[:,0], df_train[:,1], statistic='mean', bins=cuts2)
    std, edges, binnum = scipy.stats.binned_statistic(df_train[:,0], df_train[:,1], statistic='std', bins=cuts2)
    count, edges, binnum = scipy.stats.binned_statistic(df_train[:,0], df_train[:,1], statistic='count', bins=cuts2)
    
    return mean, std, count

def vectorize_variance():
    """Apply the binned variances to the original training data"""
    
    df_train, cuts1, cuts2 = ewd()
    mean, std, count = binned_statistics()
    variance=[]
    m=0
    i=0
    while i<len(count):
        maximum = count[i]

        while m < maximum:
            variance.append(std[i])
            m+=1
        else:
            m=0
            i+=1
    return np.array(variance)

def vectorize_mean():
    
    df_train, cuts1, cuts2 = ewd()
    mean, std, count = binned_statistics()
    means=[]
    m=0
    i=0
    while i<len(count):
        maximum = count[i]

        while m < maximum:
            means.append(mean[i])
            m+=1
        else:
            m=0
            i+=1
    return np.array(means)

def plot_uncertainties():
    
    df_train, cuts1, cuts2 = ewd()
    mean, std, count= binned_statistics()
    variance_vector = vectorize_variance()
    mean_vector = vectorize_mean()
    
    upper_limit = mean_vector + variance_vector
    lower_limit = mean_vector - variance_vector

   

    ## Plot the output
    plt.plot(np.linspace(-0.1, 0.1, 60), mean, label='Mean')
    plt.fill_between(df_train[:,0], upper_limit, lower_limit, color='lightblue', label='Uncertainty (std)')
    plt.legend()
    plt.title('Mixed batch of 4,000 ZLPs')
    plt.show()
    
"""Neural network functions: """
    
def custom_cost(y_true, y_pred):
    '''Chi square function'''
    return tf.reduce_mean(tf.square((y_true-y_pred)/sigma))

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

